In [0]:
table_names = []
for i in dbutils.fs.ls('mnt/jv-container/silver-zone/dbo/'):
    table_names.append(i.name.split('/')[0])
    
table_names   

['tblCity', 'tblEmps']

In [0]:
df1 = spark.read.format('delta').option('header',True).option('inferSchema',True).load('dbfs:/mnt/jv-container/silver-zone/dbo/tblCity/')
df1.show()

+-------+------------+
|pinCode|      Cities|
+-------+------------+
| 700081|   Hyderabad|
| 700082|     Chennai|
| 700083|    Banglore|
| 700084|     Lucknow|
| 700085|   New Delhi|
|     50|INVALID CITY|
+-------+------------+



In [0]:
df2 = spark.read.format('delta').option('header',True).option('inferSchema',True).load('dbfs:/mnt/jv-container/silver-zone/dbo/tblEmps/')
df2.show()

+---+-----+---+-----------+--------+-------+
| id|EName|Age|AreaPinCode|RepMgrID| salary|
+---+-----+---+-----------+--------+-------+
|  1|  AAA| 25|     700082|       0|10000.0|
|  2|  BBB| 35|     700085|       1| 8500.0|
|  3|  CCC| 40|     700081|       4| 8000.0|
|  4|  DDD| 47|     700081|       0| 7500.0|
|  5|  EEE| 30|     700081|       1| 8500.0|
|  6|  FFF| 35|     700085|       3| 7900.0|
|  7|  GGG| 36|          0|       3|    0.0|
| 10|  HHH| 20|     700083|       0|15500.0|
+---+-----+---+-----------+--------+-------+



In [0]:
df3 = df2.join(df1, df2.AreaPinCode==df1.pinCode, 'left').orderBy('id')
df3 = df3.select(df3.id,df3.EName,df3.Age,df3.RepMgrID,df3.salary,df3.pinCode,df3.Cities)
df3.show()

+---+-----+---+--------+-------+-------+---------+
| id|EName|Age|RepMgrID| salary|pinCode|   Cities|
+---+-----+---+--------+-------+-------+---------+
|  1|  AAA| 25|       0|10000.0| 700082|  Chennai|
|  2|  BBB| 35|       1| 8500.0| 700085|New Delhi|
|  3|  CCC| 40|       4| 8000.0| 700081|Hyderabad|
|  4|  DDD| 47|       0| 7500.0| 700081|Hyderabad|
|  5|  EEE| 30|       1| 8500.0| 700081|Hyderabad|
|  6|  FFF| 35|       3| 7900.0| 700085|New Delhi|
|  7|  GGG| 36|       3|    0.0|   NULL|     NULL|
| 10|  HHH| 20|       0|15500.0| 700083| Banglore|
+---+-----+---+--------+-------+-------+---------+



In [0]:
df3.write.format('delta').mode("overwrite").save('dbfs:/mnt/jv-container/gold-zone/')